#  Using Variational Graph Autoencoders for Unsupervised Learning

This tutorial demonstrates the use of Graph Auto-Encoders (GAEs) and Variational Graph Auto-Encoders (VGAEs) to perform unsupervised learning on a graph, as first implemented by <a href='https://arxiv.org/pdf/1611.07308.pdf'>Kipf (2016)</a>. The models will be implemented with PyTorch and <a href='https://pytorch-geometric.readthedocs.io/en/latest/index.html'>PyTorch Geometric</a> (a <a href='https://github.com/tkipf/gae'>tensorflow implementation</a> has been produced by the author of VGAE).
To demonstrate the suitably of the models' embeddings, we show how unsupervised training using only the reconstruction loss improves performance on a link prediction task that the model is not directly optimising for.

## Setup
Here we load required libraries, define paths to data, and define some helper functions. **Feel free to skip this section.**

In [1]:
!pip install python-igraph
!pip install torch_geometric
!pip install torch_scatter
!pip install torch-sparse
!pip install tensorboardX

import torch
import igraph as ig
import random
from datetime import datetime
from PIL import Image
from torch_geometric.nn import GCNConv, VGAE
from torch_geometric.datasets import *
from tensorboardX import SummaryWriter
from torch_geometric.nn.models.autoencoder import EPS
MAX_LOGVAR = 10
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.utils import train_test_split_edges
from torch_geometric.data import Data
from torch_geometric.nn.inits import reset
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.utils import (negative_sampling, remove_self_loops, add_self_loops)

In [2]:
# seed random number generator for reproducibility
torch.manual_seed(0)

## Load data
We'll use the PubMed dataset which is accessible via PyG.

In [3]:
# load the dataset - small download on first run
dataset_name = "PubMed"  # options: 'Citeseer', 'Cora', 'PubMed'
dataset = Planetoid(root="./datasets/"+dataset_name, name=dataset_name)
data = dataset[0]
print(data.y)

tensor([1, 1, 0,  ..., 2, 0, 2])


In [4]:
# explore key features of the graph
print('num nodes =', data.num_nodes)
print('num edges =', data.num_edges)
print('num features =', data.num_node_features)

num nodes = 19717
num edges = 88648
num features = 500


<strong>Useful PyG Data class attributes</strong>
<table width="75%">
    <header>
        <th style="text-align:left">Name</th>
        <th style="text-align:left">Attribute</th>
        <th style="text-align:left">Shape</th>
    </header>
    <tr>
        <td style="text-align:left">Node Feature Matrix</td>
        <td style="text-align:left">data.x</td>
        <td style="text-align:left">(num nodes, num features)</td>
    </tr>
    <tr>
        <td style="text-align:left">Edge Index</td>
        <td style="text-align:left">data.edge_index</td>
        <td style="text-align:left">(2, num edges)</td>
    </tr>
    <tr>
        <td style="text-align:left">Node Classes</td>
        <td style="text-align:left">data.y</td>
        <td style="text-align:left">(num nodes, )</td>
    </tr>
</table>

## Visualisation
We provide a visualisation function here. Note that the 'plot' call is skipped by default, as it is slow for larger graphs. We have rendered images of these plots in section B.3 of the paper.

In [5]:
# define a plot function using networkx
def plot(data, outfile='./graph.eps'):
    """
    plot a PyG graph using igraph
    :param data: instance of torch_geometric.data
    """
    assert data.edge_index is not None, 'cannot plot after train_test_split_edges has been called'
    g = ig.Graph()
    # set the number of vertices
    g.add_vertices(data.num_nodes)
    # get the edges as a list of tuples [(source, target) ...]
    edges = list(zip(*tuple(data.edge_index)))
    edges = [(int(a), int(b)) for a, b in edges]
    g.add_edges(edges)
    n_classes = len(np.unique(data.y))
    # randomly generate a hexadecimal colour for each class
    r = lambda: random.randint(0, 255)
    colours = ['#%02X%02X%02X' % (r(),r(),r()) for i in range(n_classes)]
    g.vs['color'] = [colours[cls] for cls in data.y]
    # save the plot to a .eps file
    ig.plot(
        g, outfile,
        layout=g.layout_auto(), vertex_size=15, vertex_colour='#D1D1D1',
        edge_curved=True, bbox=(3000, 3000), margin=20
    )
    # read and show the plot
    img = Image.open(outfile)
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img)

In [6]:
# plot the graph - ok to run on cora/citeseer but slower on pubmed/reddit etc
if data.num_nodes < 5000:
    plot(data)

## Prepare data for edge prediction

In [7]:
print(data.train_mask)
print(data.y)
print(dataset)

tensor([ True,  True,  True,  ..., False, False, False])
tensor([1, 1, 0,  ..., 2, 0, 2])
PubMed()


In [8]:
# To perform edge prediction, we need to split the graph's edges intro train and test sets
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)
# Note that data.edge_index is no longer available, as edges have been distributed between train and test
print('num train edges', data.train_pos_edge_index.shape[1])

num train edges 75352


In [9]:
# Get the device to use for training our model
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA available:', torch.cuda.is_available())

CUDA available: False


In [10]:
# We then register the relevant data to the device in use
x = data.x.to(dev)  # feature matrix
train_pos_edge_index = data.train_pos_edge_index.to(dev)  # index of true positive edges in the train set


In [11]:
print(train_pos_edge_index[0:2,0:10])
print(train_pos_edge_index.shape)
print(data.val_pos_edge_index.shape)
print(data.test_pos_edge_index.shape)

tensor([[    0,     0,     1,     1,     1,     2,     2,     3,     4,     5],
        [ 1378,  1544,  2943,  8359, 10199, 10471, 11485,  8249, 14044,  1312]])
torch.Size([2, 75352])
torch.Size([2, 2216])
torch.Size([2, 4432])


## GAE Model
To understand GAEs, we show the models as constructed by PyG. Alternatively, these can be imported directly from torch_geometric.nn
 
Importantly, the GAE class includes methods to:
<ul>
    <li>apply the encoder and decoder</li>
    <li>calculate reconstruction loss</li>
</ul>

In [12]:
# define the GAE class

class GAE(torch.nn.Module):
    r"""The Graph Auto-Encoder model from the
    `"Variational Graph Auto-Encoders" <https://arxiv.org/abs/1611.07308>`_
    paper based on user-defined encoder and decoder models.

    Args:
        encoder (Module): The encoder module.
        decoder (Module, optional): The decoder module. If set to :obj:`None`,
            will default to the
            :class:`torch_geometric.nn.models.InnerProductDecoder`.
            (default: :obj:`None`)
    """
    def __init__(self, encoder, decoder=None):
        super(GAE, self).__init__()
        self.encoder = encoder
        self.decoder = InnerProductDecoder() if decoder is None else decoder
        GAE.reset_parameters(self)

    def reset_parameters(self):
        reset(self.encoder)
        reset(self.decoder)


    def encode(self, *args, **kwargs):
        r"""Runs the encoder and computes node-wise latent variables."""
        return self.encoder(*args, **kwargs)


    def decode(self, *args, **kwargs):
        r"""Runs the decoder and computes edge probabilities."""
        return self.decoder(*args, **kwargs)


    def recon_loss(self, z, pos_edge_index):
        r"""Given latent variables :obj:`z`, computes the binary cross
        entropy loss for positive edges :obj:`pos_edge_index` and negative
        sampled edges.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            pos_edge_index (LongTensor): The positive edges to train against.
        """

        # print('recon_loss in')
        pos_loss = -torch.log(
            self.decoder(z, pos_edge_index, sigmoid=True) + EPS).mean()

        # Do not include self-loops in negative samples
        pos_edge_index, _ = remove_self_loops(pos_edge_index)
        pos_edge_index, _ = add_self_loops(pos_edge_index)

        neg_edge_index = negative_sampling(pos_edge_index, z.size(0))
        neg_loss = -torch.log(1 -
                              self.decoder(z, neg_edge_index, sigmoid=True) +
                              EPS).mean()

        # print('recon_loss out')
        return pos_loss + neg_loss


    def test(self, z, pos_edge_index, neg_edge_index):
        r"""Given latent variables :obj:`z`, positive edges
        :obj:`pos_edge_index` and negative edges :obj:`neg_edge_index`,
        computes area under the ROC curve (AUC) and average precision (AP)
        scores.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            pos_edge_index (LongTensor): The positive edges to evaluate
                against.
            neg_edge_index (LongTensor): The negative edges to evaluate
                against.
        """
        # print('test in')
        pos_y = z.new_ones(pos_edge_index.size(1))
        neg_y = z.new_zeros(neg_edge_index.size(1))
        y = torch.cat([pos_y, neg_y], dim=0)

        pos_pred = self.decoder(z, pos_edge_index, sigmoid=True)
        neg_pred = self.decoder(z, neg_edge_index, sigmoid=True)
        pred = torch.cat([pos_pred, neg_pred], dim=0)

        y, pred = y.detach().cpu().numpy(), pred.detach().cpu().numpy()

        # print('test out')
        return roc_auc_score(y, pred), average_precision_score(y, pred)


In [13]:
# define the encoder and decoder that GAE will use
# this code is slightly adapted from the PyG example to accept model_name
class GAEEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAEEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

### InnerProductDecoder
The inner product decoder from the `"Variational Graph Auto-Encoders"<https://arxiv.org/abs/1611.07308>`_ paper

$\sigma(\mathbf{Z}\mathbf{Z}^{\top})$

where $\mathbf{Z} \in \mathbb{R}^{N \times d}$ denotes the latent space produced by the encoder.

In [14]:
# To decode embeddings and attempt input reconstruction, we can use the standard InnerProductDecoder
class InnerProductDecoder(torch.nn.Module):
    r"""The inner product decoder from the `"Variational Graph Auto-Encoders"
    <https://arxiv.org/abs/1611.07308>`_ paper

    .. math::
        \sigma(\mathbf{Z}\mathbf{Z}^{\top})

    where :math:`\mathbf{Z} \in \mathbb{R}^{N \times d}` denotes the latent
    space produced by the encoder."""
    def forward(self, z, edge_index, sigmoid=True):
        r"""Decodes the latent variables :obj:`z` into edge probabilities for
        the given node-pairs :obj:`edge_index`.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            sigmoid (bool, optional): If set to :obj:`False`, does not apply
                the logistic sigmoid function to the output.
                (default: :obj:`True`)
        """
        # print('forward called')
        value = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)
        return torch.sigmoid(value) if sigmoid else value

    def forward_all(self, z, sigmoid=True):
        r"""Decodes the latent variables :obj:`z` into a probabilistic dense
        adjacency matrix.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            sigmoid (bool, optional): If set to :obj:`False`, does not apply
                the logistic sigmoid function to the output.
                (default: :obj:`True`)
        """
        print('forward_all called')
        adj = torch.matmul(z, z.t())
        return torch.sigmoid(adj) if sigmoid else adj

In [15]:
# bring the encoder and decoder together to compile the GAE
# in_channels (the dimensionality of the input data) is simply the number of features
# we define out_channels (the dimensionality of the latent vector or embedding) as 16
gae = GAE(GAEEncoder(in_channels=dataset.num_features, out_channels=16)).to(dev)

In [16]:
# define the optimizer with a learning rate of 0.01
gae_optimizer = torch.optim.Adam(gae.parameters(), lr=0.01)

In [17]:
# define a writer to log training results
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

Although we will train the GAE in an unsupervised manner (using reconstruction loss) <i>we can validate its performance by testing its ability to perform link prediction during training </i> (even though this is not used in back propagation).

In [18]:
# define a function to test our predictions against the ground truth edges
def test(model, pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [19]:
# train the model
def train_gae():
    print('training gae')
    auc, ap = None, None
    for epoch in range(0, 200):
        gae_optimizer.zero_grad()
        # get the embedding (z) of each node, shape(num_nodes, channels)
        z = gae.encode(x, train_pos_edge_index)
        # calculate the reconstruction loss to be used in backprop
        loss = gae.recon_loss(z, train_pos_edge_index)
        loss.backward()
        gae_optimizer.step()

        writer.add_scalar("loss", loss.item(), epoch)

        # calculate performance on edge prediction task
        auc, ap = test(gae, data.test_pos_edge_index, data.test_neg_edge_index)
        writer.add_scalar("AUC", auc, epoch)
        writer.add_scalar("AP", ap, epoch)
        if epoch % 100 == 0:
            print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
            print((z[train_pos_edge_index[0]] * z[train_pos_edge_index[1]]).sum(dim=1))
    return auc, ap

In [ ]:
train_gae()

training gae


/Users/SB/opt/miniconda3/envs/gnn_tutorial/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Epoch: 000, AUC: 0.8848, AP: 0.8679
tensor([0.0018, 0.0015, 0.0010,  ..., 0.0062, 0.0014, 0.0046],
       grad_fn=<SumBackward1>)


## VGAE Model
To understand VGAEs, we again show the model as constructed by PyG (torch_geometric.nn.VGAE).
 
VGAE differs from GAE in the use of two GCNs that encode the mean and variance. This allows it to reconstruct unseen inputs but requires a few updates to the main class.
Namely:
<ul>
    <li>we use a KL loss, which uses the reconstruction loss from GAEs but adds a term to measure the probability distributions</li>
    <li>the encoder uses the reparametrisation trick in order to maintain a differentiable loss function </li>
</ul>

In [ ]:
# define the VGAE class, which inherits from GAE 

class VGAE(GAE):
    r"""The Variational Graph Auto-Encoder model from the
    `"Variational Graph Auto-Encoders" <https://arxiv.org/abs/1611.07308>`_
    paper.

    Args:
        encoder (Module): The encoder module to compute :math:`\mu` and
            :math:`\log\sigma^2`.
        decoder (Module, optional): The decoder module. If set to :obj:`None`,
            will default to the
            :class:`torch_geometric.nn.models.InnerProductDecoder`.
            (default: :obj:`None`)
    """
    def __init__(self, encoder, decoder=None):
        super(VGAE, self).__init__(encoder, decoder)

    def reparametrize(self, mu, logvar):
        if self.training:
            return mu + torch.randn_like(logvar) * torch.exp(logvar)
        else:
            return mu

    def encode(self, *args, **kwargs):
        """"""
        self.__mu__, self.__logvar__ = self.encoder(*args, **kwargs)
        self.__logvar__ = self.__logvar__.clamp(max=MAX_LOGVAR)
        z = self.reparametrize(self.__mu__, self.__logvar__)
        return z

    def kl_loss(self, mu=None, logvar=None):
        r"""Computes the KL loss, either for the passed arguments :obj:`mu`
        and :obj:`logvar`, or based on latent variables from last encoding.

        Args:
            mu (Tensor, optional): The latent space for :math:`\mu`. If set to
                :obj:`None`, uses the last computation of :math:`mu`.
                (default: :obj:`None`)
            logvar (Tensor, optional): The latent space for
                :math:`\log\sigma^2`.  If set to :obj:`None`, uses the last
                computation of :math:`\log\sigma^2`.(default: :obj:`None`)
        """
        mu = self.__mu__ if mu is None else mu
        logvar = self.__logvar__ if logvar is None else logvar.clamp(
            max=MAX_LOGVAR)
        return -0.5 * torch.mean(
            torch.sum(1 + logvar - mu**2 - logvar.exp(), dim=1))

In [ ]:
# define a new encoder which incorporates new GCN layers for embedding the probability distribution
class VGAEEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VGAEEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True)
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logvar = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv_mu(x, edge_index), self.conv_logvar(x, edge_index)

In [ ]:
# bring the encoder and decoder together to compile the VGAE
# in_channels (the dimensionality of the input data) is simply the number of features
# we define out_channels (the dimensionality of the latent vector or embedding) as 16
vgae = VGAE(VGAEEncoder(in_channels=dataset.num_features, out_channels=16)).to(dev)

In [ ]:
# define a new optimizer with vgae params
vgae_optimizer = torch.optim.Adam(vgae.parameters(), lr=0.01)

In [ ]:
def train_vgae():
    print('training vgae')
    auc, ap = None, None
    for epoch in range(200):
        vgae_optimizer.zero_grad()
        # get the embedding (z) of each node, shape(num_nodes, channels)
        z = vgae.encode(x, train_pos_edge_index)
        # calculate reconstruction loss
        recon_loss = vgae.recon_loss(z, train_pos_edge_index)
        # calculate kl loss
        kl_loss = vgae.kl_loss()
        # multiply the two losses, ensuring output is nonzero
        loss = recon_loss + (1 / data.num_nodes) * kl_loss
        loss.backward()
        vgae_optimizer.step()

        writer.add_scalar("loss", loss.item(), epoch)

        # calculate performance on edge prediction task
        auc, ap = test(vgae, data.test_pos_edge_index, data.test_neg_edge_index)
        writer.add_scalar("AUC", auc, epoch)
        writer.add_scalar("AP", ap, epoch)
        if epoch % 100 == 0:
            print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    return auc, ap

In [ ]:
train_vgae()

## Experiments
We will now run each model 10 times over the dataset in order to robustly compare performance.

In [ ]:
print('running experiments on', dataset_name)
gae_results = []
vgae_results = []
n_experiments = 10
for i in range(n_experiments):
    print(i + 1)
    gae_results.append(train_gae())
    vgae_results.append(train_vgae())


In [ ]:
print('GAE results')
gae_aucs, gae_aps = list(zip(*gae_results))
print('AUC: mean = %0.3f, std = %0.3f' % (np.mean(gae_aucs), np.std(gae_aucs)))
print('AP : mean = %0.3f, std = %0.3f' % (np.mean(gae_aps), np.std(gae_aps)))

In [ ]:
print('VGAE results')
vgae_aucs, vgae_aps = list(zip(*vgae_results))
print('AUC: mean = %0.3f, std = %0.3f' % (np.mean(vgae_aucs), np.std(vgae_aucs)))
print('AP : mean = %0.3f, std = %0.3f' % (np.mean(vgae_aps), np.std(vgae_aps)))

Note the comparable AUC and means. On average, the VGAE is slightly more performant on other graphs such as Cora and Citeseer. This is likely because PubMed uses the more descriptive TF-IDF word vector, which accounts for the frequency of terms. This feature maybe more discriminative, and thus more useful when learning vertex embeddings.